<a href="https://colab.research.google.com/github/Brandon12231/780/blob/main/BOYUE%20LI%20HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Problem Introduction:
We want to determine the best purchase plan for the fertilizers to meet the grass's nutrient requirements while minimizing the total cost.
Minimize C(x,y,z)=0.01x+0.008y+0.007z
2. Decision Variables:


x
A
​
 : This represents the number of pounds of Fertilizer A we should buy.

x
B
​
 : This is the number of pounds of Fertilizer B we should purchase.

x
C
​
 : Similarly, this is for Fertilizer C.
3. Objective:
Our primary goal is to minimize the overall expenditure when buying these fertilizers. We calculate the cost by multiplying the price per pound of each fertilizer by the number of pounds we decide to buy.

4. Constraints:
Subject to:
0.02x+0.01y+0.015z≥1.2
 0.01x+0.005y+0.01z≥1.4
0.005x+0.015y+0.005z≥1.8
x≥0 y≥0 z≥0


Install Pyomo and a few optimization engines.

In [7]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre    # Installs a set of good open source optimization tools
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'    # Add to path so we can find it

import pyomo.environ as pyo         # Import pyomo

Instantiate an engine that we can use to optimize.

In [8]:
opt = pyo.SolverFactory('cbc')
  # Instantiate the 'CBC' optimization engine: one of the best open source engines that can solve
  # linear and non-linear problems with continuous and intger decision variables.

In [13]:
# Import the required library for optimization
import pyomo.environ as pyo

# --- Model Initialization ---

# Create a new optimization model
m = pyo.ConcreteModel()

# --- Decision Variables ---

# Define the variables for the amount (in pounds) of fertilizer A, B, and C we'll purchase
m.xA = pyo.Var(domain=pyo.NonNegativeReals)
m.xB = pyo.Var(domain=pyo.NonNegativeReals)
m.xC = pyo.Var(domain=pyo.NonNegativeReals)

# --- Objective Function ---

# Set our objective: minimize the total cost of purchasing the fertilizers
m.CostObj = pyo.Objective(expr=0.01*m.xA + 0.008*m.xB + 0.007*m.xC, sense=pyo.minimize)

# --- Constraints ---

# Ensure that the grass gets the required amount of each mineral
m.NitrogenConstraint = pyo.Constraint(expr=0.02*m.xA + 0.01*m.xB + 0.015*m.xC >= 1.2)
m.PhosphorusConstraint = pyo.Constraint(expr=0.01*m.xA + 0.005*m.xB + 0.01*m.xC >= 1.4)
m.PotashConstraint = pyo.Constraint(expr=0.005*m.xA + 0.015*m.xB + 0.005*m.xC >= 1.8)

# --- Solver Configuration ---

# Configure the solver we'll use (CBC in this case)
opt = pyo.SolverFactory('cbc')

# Indicate that we want to retrieve dual (shadow) values after solving
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# --- Solve the Model ---

# Solve the optimization problem and store the results
results = opt.solve(m, tee=True, keepfiles=False)

# --- Post-Processing: Display Results ---

# If the solver found an optimal solution, display it
if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    # Display the optimal amounts of each fertilizer to purchase
    print("Fertilizer A (lbs): ", m.xA.value)
    print("Fertilizer B (lbs): ", m.xB.value)
    print("Fertilizer C (lbs): ", m.xC.value)
    # Display the minimum cost achieved
    print("Total Cost ($):", m.CostObj())
    # Display the shadow prices for each constraint
    print("Nitrogen Constraint Shadow Price =", m.dual[m.NitrogenConstraint])
    print("Phosphorus Constraint Shadow Price =", m.dual[m.PhosphorusConstraint])
    print("Potash Constraint Shadow Price =", m.dual[m.PotashConstraint])
else:
    # If an optimal solution was not found, display the solver status
    print("Solver Status:", results.solver.status)


Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -printingOptions all -import /tmp/tmp459587lh.pyomo.lp -stat=1 -solve -solu /tmp/tmp459587lh.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 3 (0) rows, 3 (0) columns and 9 (0) elements
Statistics for presolved model


Problem has 3 rows, 3 columns (3 with objective) and 9 elements
Column breakdown:
3 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
3 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
0 of type L other, 0 of type Range 0.0->1.0, 0 of type Range other, 
0 of type Free 
Presolve 3 (0) rows, 3 (0) columns and 9 (0) elements
0  Obj 0 Primal inf 320 (3)
2  Obj 1.376
Optimal - object

## Extra Notes

This highlights the features that you need to complete the HW1. There are a ton more. You can learn them, as necessary, from the resources at the end.

### Abstract vs Concrete Models

There are two types of models in Pyomo: concrete and abstract. The former can be created when all your data elements (constraint parameters and objective coefficients) are available at the time of model creation. The abstract model allows us to create a model without "hard coding" data as part of it. The former is easier (hence we use it here). The latter allows us to separately specify a data file whose content will decide the model details. Abstract model is more common in some of the more popular optimization modeling languages (such as AMPL), as it allows us to focus on the type of model, without getting bogged down in the details of a specific instance.

Having said that, because Pyomo is being used within Python, we could achieve quite a bit of "program--data separation" using a concrete model too. We use ConcreteModel in this example since it's a little easier to use.

### Further Resources

1. A nice [tutorial style cookbook](https://pyomo.readthedocs.io/en/stable/) by Professor Jeffrey Kantor from Notre Dame.
1. The [official Pyomo documentation](https://pyomo.readthedocs.io/en/stable/).
